In [ ]:
import tensorflow as tf
import numpy as np
import cv2 as cv
import os

In [ ]:
# secon way to load the data 
train_path = r'C:\Users\DELL\OneDrive\Desktop\opencv\train'
val_path = r'C:\Users\DELL\OneDrive\Desktop\opencv\val'

def load_data(path,is_train=True):
    data = tf.keras.utils.image_dataset_from_directory(
        path,
        image_size=(224, 224),
        batch_size=64,
        label_mode='categorical'

    )
    class_names = data.class_names
    data = data.cache()
    if is_train:
        data = data.shuffle(buffer_size=1000)
    data = data.prefetch(buffer_size = tf.data.AUTOTUNE)

    return data,class_names

def train_val_data():
    train_data, train_class_names = load_data(train_path)
    val_data, val_class_names = load_data(val_path)

    return train_data, train_class_names, val_data
        
    
    
    
    
    

In [ ]:
train_df, train_class_names, val_df = train_val_data()

In [23]:
images = []
labels = []

for image,label in train_df.take(1):
    images.append(image.numpy())
    labels.append(label.numpy())
images = np.concatenate(images,axis=0)
labels = np.concatenate(labels,axis=0)

In [24]:
images.shape

(64, 224, 224, 3)

# ***Model Making***

In [25]:
from tensorflow.keras.applications import MobileNetV2,ResNet50V2,EfficientNetB5

In [26]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

# EfficientNetB5 Pre-trained model with freeze layers

In [28]:
base_model = EfficientNetB5(input_shape = (224,224,3),include_top=False,weights='imagenet')
base_model.trainable = False


Efficienctnetb5_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(train_class_names),activation = 'softmax')
])

In [29]:
Efficienctnetb5_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [37]:
Efficienctnetb5_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb5 (Functional)     │ (None, 7, 7, 2048)     │    28,513,527 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 14)             │         7,182 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,569,797 (112.80 MB)

 Trainable params: 2,108,942 (8.04 MB)

 Non-trainable params: 27,460,855 (104.75 MB)

In [47]:
# Efficienctnetb5_model.fit(
#     train_df,
#     validation_data=val_df,
#     epochs=10,
# )


Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1783s 8s/step - accuracy: 0.7380 - loss: 0.8286 - val_accuracy: 0.9286 - val_loss: 0.2515
Epoch 2/10
102/214 ━━━━━━━━━━━━━━━━━━━━ 15:29 8s/step - accuracy: 0.9108 - loss: 0.2732

# Model with Unfreeze layer
# using the some layer from the pre trained model

In [42]:
base_model = EfficientNetB5(input_shape = (224,224,3),include_top=False,weights='imagenet')
for layer in base_model.layers[:-20]:
    layer.trainable = False
Efficienctnetb5_model_unfreeze = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(train_class_names),activation = 'softmax')
])

In [43]:
Efficienctnetb5_model_unfreeze.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [44]:
Efficienctnetb5_model_unfreeze.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb5 (Functional)     │ (None, 7, 7, 2048)     │    28,513,527 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 14)             │         7,182 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,569,797 (112.80 MB)

 Trainable params: 6,085,262 (23.21 MB)

 Non-trainable params: 23,484,535 (89.59 MB)

In [46]:
# Efficienctnetb5_model_unfreeze.fit(
#     train_df,
#     validation_data=val_df,
#     epochs=10,
    
# )

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2249s 10s/step - accuracy: 0.6400 - loss: 1.3304 - val_accuracy: 0.8980 - val_loss: 0.2802
Epoch 2/10
  3/214 ━━━━━━━━━━━━━━━━━━━━ 38:29 11s/step - accuracy: 0.8958 - loss: 0.3571